In [1]:
suppressPackageStartupMessages({
    library(foreign) # reading stata dataset
    library(dplyr)# table manipulation
    })
# read the TIV data
mydata <- read.dta("comparison_TIV_all_resp_2.dta")
mydata <- filter(mydata,!is.na(man)) %>% select(man,fs5,spm12_mwc)
head(mydata)

,man,fs5,spm12_mwc
,<dbl>,<dbl>,<dbl>
3,1345.541,1497.891,1353.598
4,1381.081,1436.819,1359.308
5,1285.623,1268.764,1265.084
6,1574.595,1680.504,1557.601
7,1182.910,1172.865,1170.121
8,1408.842,1500.151,1391.574


In [2]:
model.mwc <- lm(spm12_mwc~man,mydata) # linear model
print(model.mwc)
sprintf("Rsquared=%f",summary(model.mwc)$r.squared) # Don't want all of summary
model.fs5 <- lm(fs5~man, mydata)
print(model.fs5)
sprintf("Rsquared=%f",summary(model.fs5)$r.squared)


Call:
lm(formula = spm12_mwc ~ man, data = mydata)

Coefficients:
(Intercept)          man  
     1.5103       0.9707  



[1] "Rsquared=0.940316"


Call:
lm(formula = fs5 ~ man, data = mydata)

Coefficients:
(Intercept)          man  
   -345.260        1.289  



[1] "Rsquared=0.460257"

In [3]:
mycleandata <- filter(mydata, fs5<3000)

In [4]:
library(broom)
library(purrr)
library(tibble)
library(rsample)
library(boot)

The familiar statistic function as required by `boot` takes a data frame and sampling indices (or weights) and returns a value or values:

In [5]:
# required arguments of boot: function (data, statistic, R)
# data : data frame to sample from
# statistic : function that calculates the statistic or statistics to bootstrap
# R : number of repetitions
diffR2 <- function(thisdata, set){
    model.mwc <- lm(spm12_mwc~man,thisdata[set,])
    model.fs5 <- lm(fs5~man, thisdata[set,])
    # By default R functions return the last value:
    summary(model.mwc)$r.squared - summary(model.fs5)$r.squared
}

The estimator for `rsamples`'s `bootstraps` function takes the permuted data and returns a tibble which contains at least the name and the estimate for each value to bootstrap:

In [6]:
diffR2est <- function(thisdata, ...){
    model.mwc <- lm(spm12_mwc~man,analysis(thisdata))
    model.fs5 <- lm(fs5~man, analysis(thisdata))
    # By default R functions return the last value:
    tibble(
        term="r2diff",
        estimate = (summary(model.mwc)$r.squared - summary(model.fs5)$r.squared),
        std.err = NA_real_
        )
}


Using `apparent=TRUE` means the sample value is also calculated, which is required for the BCa confidence interval and some other results. Unlike `boot` the value is combined with the others rather than kept separate, which makes it not as simple to use the outputs directly ourselves.

In [7]:
system.time(r2_rs <- bootstraps(mycleandata, 10000, apparent=TRUE) %>% mutate(results=map(splits,diffR2est)))
system.time(r2_boot <- boot(mycleandata, diffR2, 10000))

   user  system elapsed 
 47.824   0.113  48.626 

   user  system elapsed 
 29.680   0.034  30.191 

There's a separate function for each interval type. The `int_bca` function needs the statistic function specified again to use for Jacknife calculation.

In [8]:
system.time(r2_bca<-int_bca(r2_rs,results, .fn=diffR2est))
r2_bca

   user  system elapsed 
  2.006   0.004   2.039 

term,.lower,.estimate,.upper,.alpha,.method
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
r2diff,0.1012661,0.1394052,0.194272,0.05,BCa


`rsample` only knows how to do Jacknife, which in this case is a bit faster than the `boot` package's default approach.

In [9]:
system.time(mydataci <- boot.ci(r2_boot,type="bca"))
mydataci

   user  system elapsed 
  2.483   0.103   2.646 

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 10000 bootstrap replicates

CALL : 
boot.ci(boot.out = r2_boot, type = "bca")

Intervals : 
Level       BCa          
95%   ( 0.1008,  0.1935 )  
Calculations and Intervals on Original Scale

We can speed the `boot` package up in this case by also using Jacknife.

In [10]:
boot.ci.jack <- function(boot.out,type="bca") {
    boot.ci(boot.out=boot.out, type=type, L=empinf(boot.out=boot.out,type="jack"))
}
system.time(mydatacijack <- boot.ci.jack(r2_boot))
mydatacijack$bca

   user  system elapsed 
  0.747   0.000   0.752 

conf,,,,
0.95,384.22,9854.15,0.1007959,0.1936759


Using `rsample` for the stratification problem:

In [11]:
# Risk ratio :
armsize <- 14000
inf.vacc <- 8
inf.plac <- 86
rr <- (inf.vacc/armsize) / (inf.plac/armsize)
effic <- 1-rr
print(paste(100*c(rr, effic)))

[1] "9.30232558139535" "90.6976744186046"


In [12]:
smallarm <- 1000


In [13]:
vacctrial <- data.frame(outcome=c(rep(1,inf.plac),rep(0,smallarm-inf.plac),rep(1,inf.vacc),rep(0,smallarm-inf.vacc)), group=c(rep(0,smallarm),rep(1,smallarm)))
table(vacctrial)

       group
outcome   0   1
      0 914 992
      1  86   8

In [14]:
effrate <- function(d,set) {
   samp <- d[set,]
   place <- samp[samp$group==0,]
   treat <- samp[samp$group==1,]
   effrate <- 1 - (sum(treat$outcome)/nrow(treat))/(sum(place$outcome)/nrow(place))
   effrate
 }
effrate(vacctrial)

[1] 0.9069767

In [15]:
effrate_est <- function(d,...){
   samp <- analysis(d)
   place <- samp[samp$group==0,]
   treat <- samp[samp$group==1,]
   effrate <- 1 - (sum(treat$outcome)/nrow(treat))/(sum(place$outcome)/nrow(place))
   tibble(
       term="eff",
       estimate=effrate,
       std.err=NA_real_
       )
}

Using `boot` (single threaded):

In [16]:
system.time(boot.risk.10000 <- boot(vacctrial,effrate, 10000, strata=vacctrial$group))

   user  system elapsed 
 16.824   0.185  17.128 

Using `rsample` (single threaded) is somewhat slower:

In [17]:
system.time(effrate_rs <- bootstraps(vacctrial, 10000, apparent=TRUE) %>% mutate(results=map(splits,effrate_est)))

   user  system elapsed 
 32.114   0.070  32.361 

By default `boot` uses bootstrap replicates regression for BCa calculations. For large sample sizes this is quite slow and memory hungry.

In [18]:
system.time(vacctrial_ci <- boot.ci(boot.risk.10000,type="bca"))
vacctrial_ci

   user  system elapsed 
 71.081   0.900  73.129 

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 10000 bootstrap replicates

CALL : 
boot.ci(boot.out = boot.risk.10000, type = "bca")

Intervals : 
Level       BCa          
95%   ( 0.8140,  0.9574 )  
Calculations and Intervals on Original Scale

Here again `rsample` Jacknife for BCa calculations is faster:

In [19]:
system.time(eff_bca<-int_bca(effrate_rs,results, .fn=effrate_est))
eff_bca

   user  system elapsed 
  7.864   0.077   8.112 

term,.lower,.estimate,.upper,.alpha,.method
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
eff,0.8163446,0.9060265,0.9589057,0.05,BCa


But `boot` Jacknife for the influence estimates is also fast:

In [20]:
system.time(vacctrial_ci_jack <- boot.ci.jack(boot.risk.10000))
vacctrial_ci_jack

   user  system elapsed 
  0.747   0.046   0.810 

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 10000 bootstrap replicates

CALL : 
boot.ci(boot.out = boot.out, type = type, L = empinf(boot.out = boot.out, 
    type = "jack"))

Intervals : 
Level       BCa          
95%   ( 0.8140,  0.9574 )  
Calculations and Intervals on Original Scale

In [21]:
vacctrialbig <- data.frame(outcome=c(rep(1,inf.plac),rep(0,armsize-inf.plac),rep(1,inf.vacc),rep(0,armsize-inf.vacc)), group=c(rep(0,armsize),rep(1,armsize)))
table(vacctrialbig)

       group
outcome     0     1
      0 13914 13992
      1    86     8

It's feasible to bootstrap the larger sample size

In [22]:
system.time(effratebig_rs <- bootstraps(vacctrialbig, 10000, apparent=TRUE) %>% mutate(results=map(splits,effrate_est)))

   user  system elapsed 
355.992   1.096 360.983 

But I'm not going to run the confidence interval. Because of the Jacknife it requires more calculations than the actual bootstrap, and has larger memory requirements than the equivalent (Jacknife calculation) with `boot`.

In [23]:
## memory hungry
#system.time(effbig_bca<-int_bca(effratebig_rs,results, .fn=effrate_est))
#effbig_bca

You can do parallel processing, but it is a little more involved. You need to know what the appropriate plan is for your system (on Linux, multisession) and drop in a parallel version of `map`.

In [24]:
library(furrr)

Loading required package: future



In [25]:
plan(multisession, workers=2)
system.time(effrate_rs_par <- bootstraps(vacctrial, 10000, apparent=TRUE) %>% mutate(results=future_map(splits,effrate_est)))

   user  system elapsed 
 10.718   0.266  33.554 

In [26]:
system.time(eff_par_bca<-int_bca(effrate_rs_par,results, .fn=effrate_est))
eff_par_bca

   user  system elapsed 
  2.292   0.063   4.748 

term,.lower,.estimate,.upper,.alpha,.method
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
eff,0.8136158,0.9065003,0.9588523,0.05,BCa


In [27]:
## Might kill things
#plan(multisession, workers=2)
#system.time(effratebig_rs_par <- bootstraps(vacctrialbig, 10000, apparent=TRUE) %>% mutate(results=future_map(splits,effrate_est)))

In [28]:
#system.time(effbig_par_bca<-int_bca(effratebig_rs_par,results, .fn=effrate_est))
#effbig_bca